### Sample script for item-based collaborative filtering  
#### Execute recommendation  

#### Import libraries

In [1]:
import numpy as np
import pandas as pd

#### Parameters  

In [4]:
csv_in = 'sushi_corr-min4.csv'
# min number of common items between target user's evaluation and items in DB
min_common_items = 4
similarity = 'pearson2'

# To show more rows and columns
pd.options.display.max_rows = 999 
pd.options.display.max_columns = 999 

#### Read CSV file  

In [5]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
df.index = df.columns
print(df.shape)
print(df.info())
display(df.head())

(100, 100)
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, ebi to kaki
Data columns (total 100 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ebi             100 non-null    float64
 1   anago           100 non-null    float64
 2   maguro          100 non-null    float64
 3   ika             100 non-null    float64
 4   uni             100 non-null    float64
 5   tako            100 non-null    float64
 6   ikura           100 non-null    float64
 7   tamago          100 non-null    float64
 8   toro            99 non-null     float64
 9   amaebi          100 non-null    float64
 10  hotategai       100 non-null    float64
 11  tai             100 non-null    float64
 12  akagai          100 non-null    float64
 13  hamachi         100 non-null    float64
 14  awabi           100 non-null    float64
 15  samon           100 non-null    float64
 16  kazunoko        100 non-null    float64
 17  shako           100 non-n

,ebi,anago,maguro,ika,uni,tako,ikura,tamago,toro,amaebi,hotategai,tai,akagai,hamachi,awabi,samon,kazunoko,shako,saba,chu_toro,hirame,aji,kani,kohada,torigai,unagi,tekka_maki,kanpachi,mirugai,kappa_maki,geso,katsuo,iwashi,hokkigai,shimaaji,kanimiso,engawa,negi_toro,nattou_maki,sayori,takuwan_maki,botanebi,tobiko,inari,mentaiko,sarada,suzuki,tarabagani,ume_shiso_maki,komochi_konbu,tarako,sazae,aoyagi,toro_samon,sanma,hamo,nasu,shirauo,nattou,ankimo,kanpyo_maki,negi_toro_maki,gyusashi,hamaguri,basashi,fugu,tsubugai,ana_kyu_maki,hiragai,okura,ume_maki,sarada_maki,mentaiko_maki,buri,shiso_maki,ika_nattou,zuke,himo,kaiware,kurumaebi,mekabu,kue,sawara,sasami,kujira,kamo,himo_kyu_maki,tobiuo,ishigakidai,mamakari,hoya,battera,kyabia,karasumi,uni_kurage,karei,hiramasa,namako,shishamo,kaki
ebi,1.000000,0.041670,0.021203,0.206436,0.022451,0.198265,-0.026239,0.099235,-0.087487,0.323030,0.188516,0.004619,0.115870,-0.050579,0.083678,0.069446,0.107151,0.192423,-0.064315,-0.003323,0.029503,-0.002914,0.347253,0.058854,0.111497,0.141773,0.082542,-0.054532,0.073161,0.112095,0.294848,0.169927,-0.036643,0.140297,-0.043168,0.086153,0.062242,-0.043346,0.090278,0.031343,0.057652,0.399100,0.066609,-0.251303,0.071525,0.038575,0.026707,0.427351,0.087659,0.064040,0.205701,0.146593,0.082756,-0.038730,0.079343,0.113050,0.102810,0.386408,0.008712,0.179260,0.148251,0.001676,-0.275201,0.025000,0.035218,-6.394853e-02,-0.053416,-0.209635,0.205904,0.264104,-0.041636,0.091952,0.174842,-0.045881,0.146580,0.187702,-0.023904,-0.029762,0.254210,0.373586,-0.010466,0.128575,-0.134364,0.167836,0.163357,1.216661e-01,0.229450,1.254319e-01,0.343765,0.157644,0.058099,-0.436666,0.062678,0.055212,-0.089935,-0.103449,0.018913,0.109100,-0.041196,0.310555
anago,0.041670,1.000000,0.113932,0.016100,0.187935,0.076043,0.021129,0.112136,0.099133,0.110108,0.206372,0.138536,0.143371,0.154033,0.217419,0.167719,0.068855,0.319963,0.274743,0.160865,0.262307,0.256871,0.055260,0.312371,0.188373,0.607857,0.245864,0.213538,0.247184,-0.051375,0.105479,0.169058,0.289753,0.059714,0.317122,0.283326,0.269195,0.086922,-0.111390,0.342368,0.048797,0.222668,0.165554,0.354744,0.057793,-0.022386,0.268273,0.125440,-0.027181,0.049501,0.140057,0.115673,0.355402,0.197924,0.163319,0.413881,0.028479,0.216590,-0.133426,0.084352,0.062612,-0.027188,0.000952,0.235844,0.350964,-1.415985e-01,0.494637,0.553621,0.123473,-0.118322,0.082120,0.073974,0.585985,0.535435,0.296459,-0.071659,-0.052126,0.470336,0.071989,0.180006,0.186567,-0.045918,0.215549,-0.433013,0.092219,3.551933e-17,-0.238851,3.760070e-01,-0.023613,0.087706,0.073494,0.585369,0.595696,0.202116,-0.435366,0.382834,-0.443351,0.103367,0.279977,-0.381397
maguro,0.021203,0.113932,1.000000,0.178068,0.056439,0.026593,0.050876,-0.103009,0.285670,0.065339,0.056172,0.252269,0.012550,0.362685,-0.007964,0.240347,0.064959,-0.031159,0.178099,0.503102,0.283229,0.302277,0.070093,0.033829,-0.061046,0.071035,0.578965,0.285328,0.057216,0.195450,0.047829,0.357937,0.058690,0.063841,0.202229,-0.021501,-0.106276,0.270908,-0.010999,0.057906,0.007248,0.060220,0.160261,-0.019883,0.093952,-0.046215,-0.082923,0.027611,0.031366,0.186760,0.078556,-0.165850,0.068093,0.235932,0.330118,0.399444,0.044225,0.000664,-0.053954,-0.065994,0.046289,0.546487,0.368289,-0.059453,0.321651,1.695335e-16,-0.202410,-0.129785,0.206257,-0.138591,0.102040,-0.036487,0.298653,0.188106,-0.040874,0.009840,0.439573,-0.267729,0.035195,-0.105673,0.280102,-0.082014,0.211303,0.298389,0.050008,3.760333e-01,0.183254,-2.643693e-01,-0.175817,0.374431,-0.035006,0.044473,-0.136931,-0.214903,0.307310,0.110350,0.140150,-0.161639,-0.031623,0.458225
ika,0.206436,0.016100,0.178068,1.000000,0.094027,0.562615,-0.032448,-0.077488,0.146611,0.219407,0.275457,0.115335,0.307476,0.129432,0.185109,-0.001308,0.089798,0.138057,0.049930,0.055386,0.172705,0.150660,0.195965,0.008049,0.280947,0.066275,0.077112,0.208994,0.184287,0.142120,0.410577,0.187486,0.215691,0.254073,0.203636,0.018605,0.128805,0.094396,0.101881,0.024135,0.195277,0.326057,

In [6]:
if similarity == 'pearson2':
    df = (df+1.0)/2
    print(df.shape)  
    display(df.head()) 


(100, 100)


,ebi,anago,maguro,ika,uni,tako,ikura,tamago,toro,amaebi,hotategai,tai,akagai,hamachi,awabi,samon,kazunoko,shako,saba,chu_toro,hirame,aji,kani,kohada,torigai,unagi,tekka_maki,kanpachi,mirugai,kappa_maki,geso,katsuo,iwashi,hokkigai,shimaaji,kanimiso,engawa,negi_toro,nattou_maki,sayori,takuwan_maki,botanebi,tobiko,inari,mentaiko,sarada,suzuki,tarabagani,ume_shiso_maki,komochi_konbu,tarako,sazae,aoyagi,toro_samon,sanma,hamo,nasu,shirauo,nattou,ankimo,kanpyo_maki,negi_toro_maki,gyusashi,hamaguri,basashi,fugu,tsubugai,ana_kyu_maki,hiragai,okura,ume_maki,sarada_maki,mentaiko_maki,buri,shiso_maki,ika_nattou,zuke,himo,kaiware,kurumaebi,mekabu,kue,sawara,sasami,kujira,kamo,himo_kyu_maki,tobiuo,ishigakidai,mamakari,hoya,battera,kyabia,karasumi,uni_kurage,karei,hiramasa,namako,shishamo,kaki
ebi,1.000000,0.520835,0.510601,0.603218,0.511225,0.599132,0.486880,0.549618,0.456256,0.661515,0.594258,0.502309,0.557935,0.474710,0.541839,0.534723,0.553576,0.596212,0.467843,0.498339,0.514751,0.498543,0.673626,0.529427,0.555748,0.570886,0.541271,0.472734,0.536580,0.556048,0.647424,0.584964,0.481679,0.570149,0.478416,0.543076,0.531121,0.478327,0.545139,0.515672,0.528826,0.699550,0.533305,0.374348,0.535763,0.519288,0.513353,0.713676,0.543830,0.532020,0.602851,0.573297,0.541378,0.480635,0.539672,0.556525,0.551405,0.693204,0.504356,0.589630,0.574126,0.500838,0.362399,0.512500,0.517609,0.468026,0.473292,0.395182,0.602952,0.632052,0.479182,0.545976,0.587421,0.477059,0.573290,0.593851,0.488048,0.485119,0.627105,0.686793,0.494767,0.564287,0.432818,0.583918,0.581678,0.560833,0.614725,0.562716,0.671882,0.578822,0.529049,0.281667,0.531339,0.527606,0.455033,0.448275,0.509457,0.554550,0.479402,0.655278
anago,0.520835,1.000000,0.556966,0.508050,0.593967,0.538021,0.510565,0.556068,0.549567,0.555054,0.603186,0.569268,0.571685,0.577017,0.608710,0.583859,0.534427,0.659981,0.637371,0.580433,0.631153,0.628435,0.527630,0.656186,0.594187,0.803929,0.622932,0.606769,0.623592,0.474312,0.552739,0.584529,0.644876,0.529857,0.658561,0.641663,0.634597,0.543461,0.444305,0.671184,0.524398,0.611334,0.582777,0.677372,0.528896,0.488807,0.634137,0.562720,0.486410,0.524750,0.570028,0.557836,0.677701,0.598962,0.581660,0.706941,0.514239,0.608295,0.433287,0.542176,0.531306,0.486406,0.500476,0.617922,0.675482,0.429201,0.747318,0.776811,0.561736,0.440839,0.541060,0.536987,0.792993,0.767717,0.648230,0.464170,0.473937,0.735168,0.535995,0.590003,0.593283,0.477041,0.607775,0.283494,0.546110,0.500000,0.380574,0.688003,0.488194,0.543853,0.536747,0.792685,0.797848,0.601058,0.282317,0.691417,0.278324,0.551683,0.639988,0.309301
maguro,0.510601,0.556966,1.000000,0.589034,0.528220,0.513296,0.525438,0.448495,0.642835,0.532670,0.528086,0.626134,0.506275,0.681343,0.496018,0.620173,0.532480,0.484420,0.589050,0.751551,0.641615,0.651138,0.535046,0.516915,0.469477,0.535518,0.789482,0.642664,0.528608,0.597725,0.523914,0.678968,0.529345,0.531921,0.601115,0.489249,0.446862,0.635454,0.494501,0.528953,0.503624,0.530110,0.580130,0.490059,0.546976,0.476893,0.458539,0.513806,0.515683,0.593380,0.539278,0.417075,0.534047,0.617966,0.665059,0.699722,0.522112,0.500332,0.473023,0.467003,0.523144,0.773244,0.684145,0.470274,0.660825,0.500000,0.398795,0.435107,0.603129,0.430705,0.551020,0.481757,0.649327,0.594053,0.479563,0.504920,0.719786,0.366135,0.517597,0.447164,0.640051,0.458993,0.605651,0.649195,0.525004,0.688017,0.591627,0.367815,0.412091,0.687215,0.482497,0.522237,0.431535,0.392549,0.653655,0.555175,0.570075,0.419180,0.484189,0.729113
ika,0.603218,0.508050,0.589034,1.000000,0.547013,0.781308,0.483776,0.461256,0.573305,0.609704,0.637728,0.557667,0.653738,0.564716,0.592554,0.499346,0.544899,0.569028,0.524965,0.527693,0.586353,0.575330,0.597982,0.504024,0.640473,0.533137,0.538556,0.604497,0.592144,0.571060,0.705288,0.593743,0.607845,0.627037,0.601818,0.509302,0.564403,0.547198,0.550940,0.512067,0.597638,0.663029,0.619528,0.453856,0.555242,0.566588,0.608529,0.485806,0.634048,0.647482,0.573723,0.581312,0.638877,0.532822,0.5

In [7]:
def predict_scores(df_sim, ser_target):
    ret = {}
    for item1 in df_sim.index:  # not yet rated by the target user
        v1 = df_sim.loc[item1]
        #if v1.isnull().sum() > 0:  # debug
        #    print('v1:',v1)  # debug
        if v1.notnull().sum() < min_common_items: continue
        v11 = v1[ v1.notnull() ]
        t11 = ser_target[ v1.notnull() ]
        pred1 = (v11 * t11).sum() / np.abs(v11).sum()
        #print('v11:',v11)  # debug
        #print('t11:',t11)  # debug
        #print('pred1:',pred1)  # debug
        ret[item1] = pred1
    
    ser_ret = pd.Series(ret)
    
    return ser_ret.sort_values(ascending=False)

Function for user-based collaborative filtering.  

arguments: dictionary of scores for the target user  
and the number of items to recommend.  

ex)
```
get_recomm_by_user_sim(df, {'maguro':1, 'ika':1, 'uni':3,
                        'awabi':4, 'hirame':4, 'aoyagi':4})  
```
-> return list such as [('akagai', 2.9835603009918303), ('mirugai', 2.945676429588114), ...]

In [8]:
def get_recomm_by_item_sim(df, target_dic):
    ser_target = pd.Series(target_dic)
    #print(target_dic)  # debug
    #print(ser_target)  # debug
    # make dataframe with columns included in target_dic
    #print(df.shape)  # debug
    df_scores = df[ ser_target.index ]
    #print(df_scores.shape)  # debug
    # drop rows included in target_dic (already rated)
    df_scores = df_scores.drop(index=ser_target.index)
    #print(df_scores.shape)  # debug
    #display(df_scores.head())  # debug
    recomm = predict_scores(df_scores, ser_target)
    
    return recomm

#### Do recommendation  

In [9]:
recomm = get_recomm_by_item_sim(df,
                                {'maguro':1, 'ika':1, 'uni':3,
                                 'awabi':4, 'hirame':4, 'aoyagi':4})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

recomm = get_recomm_by_item_sim(df,
                                {'anago': 3, 'maguro': 4, 'ikura': 3,
                                 'hamachi': 4, 'samon': 4, 'unagi': 3,
                                 'suzuki': 2, 'hamo': 1, 'nasu': 1,
                                 'shiso_maki': 2})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

Number of items calculated: 94
Recommendation:
kohada          3.782942
sayori          3.552994
ana_kyu_maki    3.510626
anago           3.498714
nattou_maki     3.498144
dtype: float64
Number of items calculated: 90
Recommendation:
sawara      3.603589
mamakari    3.355246
kamo        3.178161
battera     2.962333
chu_toro    2.957528
dtype: float64
